##  Deal Smart Shopper

#### This script is an example of a **Agent**.
#### The agent doesn't just check price, but evaluates value. 
- It interprets "good game".
- It uses a "Review Check" simulation (LLM Reasoning)
- It Loops back if the first result is bad. 

In [1]:
import operator
import requests
import os
from dotenv import load_dotenv
from typing import TypedDict, List, Annotated, Dict
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, BaseMessage
from langgraph.graph import StateGraph, END

In [2]:
# Load Env file
load_dotenv()

# Pass API Key to the model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0 
    )


In [3]:
# The STATE

class AgentState(TypedDict):
    query: str
    max_price: float
    attempts: int
    messages: Annotated[
        List[BaseMessage],
        operator.add
    ]

In [4]:
# Helper to map Store IDs (e.g. "1") to Names (e.g. "Steam")
STORE_MAP = {}

def get_store_name(store_id: str) -> str:
    """ Fetches and caches store names from CheapShark """
    global STORE_MAP

    if not STORE_MAP:
        try:
            response = requests.get("https://cheapshark.com/api/1.0/stores")
            response.raise_for_status()
            data = response.json()
            STORE_MAP = {store["storeID"]: store["storeName"] for store in data}
        except Exception as e:
            print(f"Error fetching store names: {e}")
            STORE_MAP = {}

    return STORE_MAP.get(str(store_id), f"Store {store_id}")

In [5]:
def search_deal_tool(game_name: str) -> str:
    """
    Search CheapShark for a game and returns the best deals
    """
    
    print(f"Searching CheapShark for {game_name}")

    try:
        # step A: Find the Game ID
        search_url = f"https://www.cheapshark.com/api/1.0/games?title={game_name}&limit=1"
        search_resp = requests.get(search_url)
        search_resp.raise_for_status()
        search_data = search_resp.json()

        if not search_data:
            raise ValueError(f"No results found for {game_name}")

        game_id = search_data[0]['gameID']
        official_title = search_data[0]['external']
        print(f"Found Game ID: {game_id}")

        # step B: Get the deals for this game
        lookup_url = f"https://www.cheapshark.com/api/1.0/games?id={game_id}"
        deals_resp = requests.get(lookup_url)
        deals_resp.raise_for_status()
        deals_data = deals_resp.json()

        # step C: Parse the Top 3 Deals
        deals = deals_data.get("deals", [])
        if not deals:
            raise ValueError(f"No deals found for {game_name}")

        # Sort by price and take top 3
        deals.sort(key=lambda x: float(x['price']))
        
        # Format the outout for LLM
        output_lines = [f"Found Game: {official_title}"]

        for deal in deals[:3]:
            store_name = get_store_name(deal['storeID'])
            price = deal['price']
            retail = deal['retailPrice']
            savings = ""
            if float(deal["savings"]) > 0:
                savings = f"(Retail: ${retail})"
            output_lines.append(f"- ${price} at {store_name} {savings}")

        return "\n".join(output_lines)
        
    except Exception as e:
        print(f"Error searching for {game_name}: {str(e)}")
        return f"Error searching for {game_name}: {str(e)}"


### The NODES - Reasoner, Executor

In [6]:
# Node 1 - Reasoner Node

def reasoner_node(state: AgentState):
    """The Brain: Gemini 2.5 Flash decides the next step."""
    print("--- 🧠 GEMINI REASONING ---")
    
    sys_msg = SystemMessage(content="""
    You are a Smart Deal Hunter. 
    Your goal: Find a game deal that matches the user's query and max price.

    1. Call the Search Tool to find real prices.
    2. Analyze the Tool Result:
       - If the price is <= max_price: ACCEPT.
       - If the price is > max_price: REJECT.
       - If the tool says "Game not found": Try a variation of the name or GIVEUP.
    
    Reply with STRICTLY one format:
    - SEARCH: <game_name>
    - FOUND: <game_name> is available for $<price> at <store>.
    - RETRY: <new_search_term>
    - GIVEUP: No deals found under budget.
    """)
    
    # Simple prompt engineering to force structured thought
    user_input = f"User Request: Find '{state['query']}' for under ${state['max_price']}."
    messages = [sys_msg, HumanMessage(content=user_input)] + state['messages']
    
    response = llm.invoke(messages)
    return {"messages": [response]}


In [7]:
# Node 2 - Executor

def executor_node(state: AgentState):
    """The Hand: Executes the real API call."""
    last_msg = state['messages'][-1].content
    
    # Parse Command
    if ":" in last_msg:
        command, target = last_msg.split(":", 1)
        target = target.strip()
    else:
        # Fallback if Gemini chats instead of commanding
        return {} 
    
    if command == "SEARCH" or command == "RETRY":
        # CALL THE REAL API
        result_text = search_deal_tool(target)
        
        # Feed result back to Gemini as a HumanMessage (simulating tool output)
        return {
            "messages": [HumanMessage(content=result_text)],
            "attempts": state["attempts"] + 1
        }
    
    return {}
    
    

In [8]:
# Graph Construction

workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("reasoner", reasoner_node)
workflow.add_node("executor", executor_node)

In [9]:
# Add edges
workflow.set_entry_point("reasoner")

def router(state: AgentState):
    # Safety Circuit Breaker
    if state["attempts"] >= 3:
        return "end"

    last_msg = state["messages"][-1].content

    if "SEARCH:" in last_msg or "RETRY:" in last_msg:
        return "execute"
    elif "FOUND:" in last_msg or "GIVEUP:" in last_msg:
        return "end"
    else:
        return "execute" # Fallback to retry loop

workflow.add_conditional_edges(
    "reasoner",
    router,
    {
        "execute": "executor",
        "end": END
    }
)

workflow.add_edge("executor", "reasoner")

In [10]:
# Compile

app = workflow.compile()

In [11]:
# Real life simulation

# SCENARIO 1: A Realistic Request (Batman is usually cheap)
print("\n=== RUN 1: Searching for 'Batman Arkham Knight' under $5 ===")
result = app.invoke({
    "query": "Batman Arkham Knight", 
    "max_price": 5.00, 
    "attempts": 0, 
    "messages": []
})
print(f"\nFINAL OUTPUT: {result['messages'][-1].content}")


=== RUN 1: Searching for 'Batman Arkham Knight' under $5 ===
--- 🧠 GEMINI REASONING ---
Searching CheapShark for Batman Arkham Knight
Found Game ID: 107598
--- 🧠 GEMINI REASONING ---

FINAL OUTPUT: FOUND: Batman: Arkham Knight is available for $3.99 at GOG.


In [12]:
# SCENARIO 2: An Impossible Request (New AAA game for cheap)
print("\n=== RUN 2: Searching for 'Elden Ring' under $10 ===")
result = app.invoke({
    "query": "Elden Ring", 
    "max_price": 10.00, 
    "attempts": 0, 
    "messages": []
})
print(f"\nFINAL OUTPUT: {result['messages'][-1].content}")


=== RUN 2: Searching for 'Elden Ring' under $10 ===
--- 🧠 GEMINI REASONING ---
Searching CheapShark for Elden Ring
Found Game ID: 236717
--- 🧠 GEMINI REASONING ---
--- 🧠 GEMINI REASONING ---
--- 🧠 GEMINI REASONING ---
--- 🧠 GEMINI REASONING ---

FINAL OUTPUT: GIVEUP: No deals found under budget.
